In [1]:
import os
import datetime
import time
from zipfile import ZipFile
import json
from pathlib import Path
from pprint import pprint
import tomli
import geopandas as gpd
import shapely
import requests
from requests.auth import HTTPBasicAuth
from planet import api
from planet import filters
from shapely.geometry import shape

C:\Users\mnowatz\AppData\Local\Temp\ipykernel_50596\1962284598.py:13: ClientV1DeprecationWarning: The planet.api module is deprecated and will be removed in version 2.0.0. For more details please see the discussion at https://github.com/planetlabs/planet-client-python/discussions.
  from planet import api


In [2]:
with open("config.toml", "rb") as f:
    config = tomli.load(f)
API_KEY = config['keys']['api_key']
API_KEY != None

True

In [3]:
client = api.ClientV1(api_key=API_KEY)

In [49]:
aoi_geom = gpd.read_file("data/accola_boundary_poly.geojson")
aoi_geom = aoi_geom.explode()
aoi_geom = aoi_geom.to_json()
aoi_geom = json.loads(aoi_geom)
aoi_coords = aoi_geom['features'][0]['geometry']
test_start_date = datetime.datetime(year=2019,month=6,day=1)
test_stop_date = datetime.datetime(year=2019,month=6,day=30)

In [38]:
# create an api request from the search specifications
def build_request(aoi_geom, start_date, stop_date):
    '''build a data api search request for clear PSScene 4-Band imagery'''
    item_type = 'PSScene'
    query = filters.and_filter(
        filters.geom_filter(aoi_geom),
        filters.range_filter('clear_percent', gte=90),
        filters.date_range('acquired', gt=start_date),
        filters.date_range('acquired', lt=stop_date)
    )
    return filters.build_search_request(query, ['PSScene'])

In [64]:
request = build_request(aoi_coords, test_start_date, test_stop_date)
request

{'item_types': ['PSScene'],
 'filter': {'type': 'AndFilter',
  'config': ({'field_name': 'geometry',
    'type': 'GeometryFilter',
    'config': {'type': 'Polygon',
     'coordinates': [[[-93.68086364941513, 41.98756192166665],
       [-93.6528361843452, 41.98776639086707],
       [-93.65305262630085, 41.97616375402928],
       [-93.68074095705796, 41.97604456369364],
       [-93.68086364941513, 41.98756192166665]]]}},
   {'field_name': 'clear_percent',
    'type': 'RangeFilter',
    'config': {'gte': 90}},
   {'field_name': 'acquired',
    'type': 'DateRangeFilter',
    'config': {'gt': '2019-06-01T00:00:00Z'}},
   {'field_name': 'acquired',
    'type': 'DateRangeFilter',
    'config': {'lt': '2019-06-30T00:00:00Z'}})}}

In [65]:
# search the data api
def search_data_api(request, client, limit=500):
    result = client.quick_search(request)
    
    # this returns a generator
    return result.items_iter(limit=limit)

items = list(search_data_api(request, client))
print(len(items))

15


In [1]:
items

NameError: name 'items' is not defined

In [66]:
test_items = items[:2]
# filter to item ids
ids = [i['id'] for i in test_items]
ids

['20190629_153143_1020', '20190629_164329_1024']

In [67]:
item_type = 'PSScene'
# 'analytic_sr_udm2' is the basic surface reflectance corrected image, 4 bands
bundle = 'analytic_sr_udm2'
# and the 8 band version
# bundle = 'analytic_8b_sr_udm2'

In [70]:
# specify tools

# clip to AOI
clip_tool = {'clip': {'aoi': aoi_coords}}

# convert to NDVI
bandmath_tool = {'bandmath': {
    "pixel_type": "32R",
    "b1": "(b4 - b3) / (b4+b3)"
}}

tools = [clip_tool, bandmath_tool]
pprint(tools)

[{'clip': {'aoi': {'coordinates': [[[-93.68086364941513, 41.98756192166665],
                                    [-93.6528361843452, 41.98776639086707],
                                    [-93.65305262630085, 41.97616375402928],
                                    [-93.68074095705796, 41.97604456369364],
                                    [-93.68086364941513, 41.98756192166665]]],
                   'type': 'Polygon'}}},
 {'bandmath': {'b1': '(b4 - b3) / (b4+b3)', 'pixel_type': '32R'}}]


In [74]:
name = 'tutorial_order'

orders_request = {
    'name': name,
    'products': [{
        'item_ids': ids,
        'item_type': item_type,
        'product_bundle': bundle
    }],
    'tools': tools,
    'delivery': {
        'single_archive': True,
        'archive_filename':'{{name}}_{{order_id}}.zip',
        'archive_type':'zip'
    },
        'notifications': {
                   'email': False
    },
}

# pprint(orders_request, indent=1)

In [75]:
order_info = client.create_order(orders_request).get()

order_id = order_info['id']
order_id

'4fc4e8f9-58e4-4521-966d-56a34ee41797'

In [87]:
order_info['_links']['_self']

'https://api.planet.com/compute/ops/orders/v2/4fc4e8f9-58e4-4521-966d-56a34ee41797'

In [77]:
def poll_for_success(order_id, client, num_loops=50) -> None:
    count = 0
    while(count < num_loops):
        count += 1
        order_info = client.get_individual_order(order_id).get()
        state = order_info['state']
        print(state)
        success_states = ['success', 'partial']
        if state == 'failed':
            raise Exception(response)
        elif state in success_states:
            break
        
        time.sleep(30)
        
poll_for_success(order_id, client)

success


In [78]:
demo_data_dir = os.path.join('data', 'demo')
# make the download directory if it doesn't exist
Path(demo_data_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
orders_url = order_info['_links']['_self']

In [ ]:
def download_order(order_url, auth, overwrite=False):
    r = requests.get(order_url, auth=auth)
    print(r)

    response = r.json()
    results = response['_links']['results']
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    results_paths = [pathlib.Path(os.path.join('data', n)) for n in results_names]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name, path in zip(results_urls, results_names, results_paths):
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))
            
    return dict(zip(results_names, results_paths))

In [89]:
%system planet orders download 4fc4e8f9-58e4-4521-966d-56a34ee41797

['{"_links": {"_self": "https://api.planet.com/compute/ops/orders/v2/4fc4e8f9-58e4-4521-966d-56a34ee41797", "results": [{"delivery": "success", "expires_at": "2022-09-28T18:05:21.663Z", "location": "https://api.planet.com/compute/ops/download/?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE2NjQzODgzMjEsInN1YiI6ImZPalVuaFpvL0YzRWhkbGpWVG85K0ExNVFEOHFwQTMrTWZPUzFnVDNRSzBNcXBVWFI0VGNEY0tkRmRCOFU3TXZEUHBVSTJqekNwd2lycDRzUkhNQlZRPT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiIiLCJhc3NldHMiOlt7Iml0ZW1fdHlwZSI6IiIsImFzc2V0X3R5cGUiOiIiLCJpdGVtX2lkIjoiIn1dLCJ1cmwiOiJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY29tcHV0ZS1vcmRlcnMtbGl2ZS80ZmM0ZThmOS01OGU0LTQ1MjEtOTY2ZC01NmEzNGVlNDE3OTcvdHV0b3JpYWxfb3JkZXJfNGZjNGU4ZjktNThlNC00NTIxLTk2NmQtNTZhMzRlZTQxNzk3LnppcD9FeHBpcmVzPTE2NjQzODgzMjFcdTAwMjZHb29nbGVBY2Nlc3NJZD1jb21wdXRlLWdjcy1zdmNhY2MlNDBwbGFuZXQtY29tcHV0ZS1wcm9kLmlhbS5nc2VydmljZWFjY291bnQuY29tXHUwMDI2U2lnbmF0dXJlPWYxWGREZE5vbHF3Q3ZwViUyQjZBT1FMYlZobzB4TmFGbiUyQjlFMlJtb0RnUklLO